In [10]:
import requests, json, time
env = json.load(open("env.json"))
"""
env.json
[
    "BSCscan": bscscan apikey,
    "FTMscan": ftmscan apikey,
    "SNOWtrace": snowtrace apikey
]
"""
proxy = False
proxies = {
    "http":"192.168.122.6:3128",
    "https":"192.168.122.6:3128",
    "ftp":"192.168.122.6:3128"
}

In [34]:
tokenList = [
    {
        "chain":"BSC",
        "token":"TAC",
        "contract":"0x6adbaa7c9ed1f29d647e37b1970e32bb41fb18ab",
        "decimal":9,
        "ids":"taichi"
    },
    {
        "chain":"BSC",
        "token":"XEUS",
        "contract":"0x52eb4a4af85bf7921813a12b7cc9ec02b5e94d6e",
        "decimal":9,
        "ids":"xeus"
    },
    {
        "chain":"FTM",
        "token":"SPA",
        "contract":"0x8e2549225E21B1Da105563D419d5689b80343E01",
        "decimal":9,
        "ids":"spartacus"
    },
    {
        "chain":"FTM",
        "token":"HEC",
        "contract":"0x75bdef24285013387a47775828bec90b91ca9a5f",
        "decimal":9,
        "ids":"hector-dao"
    },
    {
        "chain":"FTM",
        "token":"EXOD",
        "contract":"0x8de250c65636ef02a75e4999890c91cecd38d03d",
        "decimal":9,
        "ids":"exodia"
    },
    {
        "chain":"AVAX",
        "token":"TIME",
        "contract":"0x136acd46c134e8269052c62a67042d6bdedde3c9",
        "decimal":9,
        "ids":"wonderland"
    },
    {
        "chain":"AVAX",
        "token":"SB",
        "contract":"0xE9Eb40d52CE4744322204d4a29Af63C30f0260a4",
        "decimal":9,
        "ids":"snowbank"
    },
    {
        "chain":"ONE",
        "token":"RVRS",
        "contract":"0xE9Eb40d52CE4744322204d4a29Af63C30f0260a4",
        "decimal":9,
        "numbers":8.95,
        "staking":266600/100,
        "starting":1638032419,
        "ids":"reverse-protocol"
    },
    {
        "chain":"ONE",
        "token":"ODAO",
        "contract":"0xE9Eb40d52CE4744322204d4a29Af63C30f0260a4",
        "decimal":9,
        "numbers":0.003308,
        "staking":1_400_000_000/100,
        "starting":1638032419,
        "ids":"onedao-finance"
    },
]

In [41]:
result = 0
for token in tokenList:
    if token["chain"] == "ONE":
        numbers = token["numbers"]*(token["staking"]**((time.time()-token["starting"])/(60*60*24*365)))
    else:
        if token["chain"] == "BSC":
            url = "https://api.bscscan.com/api"
            api = env["BSCscan"]
        elif token["chain"] == "FTM":
            url = "https://api.ftmscan.com/api"
            api = env["FTMscan"]
        else:
            url = "https://api.snowtrace.io/api"
            api = env["SNOWtrace"]

        dico = {"module":"account",
            "action":"tokenbalance",
            "contractaddress":token["contract"],
            "address":"0x97A0bD28796621FAa6d168ee414B814b307517FF",
            "tag":"latest",
            "apikey":api}
        if proxy:
            r = requests.get(url, params=dico, proxies=proxies)
        else:
            r = requests.get(url, params=dico)
        numbers = int(r.json()["result"])/(10**token["decimal"])
        if token["ids"] == "snowbank":
            numbers = numbers*2/3
    dico = {
        "ids":token["ids"],
        "vs_currencies":"usd"
    }
    r = requests.get(
        "https://api.coingecko.com/api/v3/simple/price", 
        params=dico
    )
    price = r.json()[token["ids"]]["usd"]
    tt = numbers*price
    result += tt
    print(f"""{token["token"]} : {numbers} at price {price} egal {tt}""")
print()
print(f"Total : {result} $")


TAC : 0.156696087 at price 107.87 egal 16.902806904690003
XEUS : 0.072690848 at price 327.37 egal 23.79680290976
SPA : 0.271823866 at price 168.54 egal 45.81319437564
HEC : 0.134131092 at price 217.04 egal 29.11181220768
EXOD : 0.008305197 at price 2088.45 egal 17.34498867465
TIME : 0.007166967 at price 7576.01 egal 54.29701366167
SB : 0.023761278666666667 at price 729.46 egal 17.332902336186667
RVRS : 8.953853054789827 at price 1.15 egal 10.2969310130083
ODAO : 0.0033109716716588394 at price 8741.19 egal 28.94183246658753

Total : 243.83828454987253 $


In [42]:
repartition = [
    ("4T", 0.2),
    ("Gouillou", 0.14),
    ("Couillet", 0.14),
    ("Synold", 0.14),
    ("Rigot", 0.14),
    ("Djafri", 0.11),
    ("Ferro", 0.09),
    ("Pot", 0.03),
]
for name, taux in repartition:
    print(f"{name} : {taux*result} $")

4T : 48.767656909974505 $
Gouillou : 34.13735983698216 $
Couillet : 34.13735983698216 $
Synold : 34.13735983698216 $
Rigot : 34.13735983698216 $
Djafri : 26.82221130048598 $
Ferro : 21.945445609488527 $
Pot : 7.315148536496175 $
